In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchvision.transforms import v2

[MRI Scans Alzeimer Detection Dataset (via Hugging Face)](https://huggingface.co/datasets/yogitamakkar178/mri_scans_alzeimer_detection)

In [2]:
from datasets import load_dataset, concatenate_datasets

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("yogitamakkar178/mri_scans_alzeimer_detection")

Resolving data files:   0%|          | 0/9552 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1279 [00:00<?, ?it/s]

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5], std=[0.5])
])
def preprocess(data):
    data["image"] = [transform(image) for image in data["image"]]
    return data
ds = ds.with_format("torch") # ds = ds.with_format("torch", device=device)
ds = ds.with_transform(preprocess)

ds = concatenate_datasets([ds["train"], ds["test"]]).shuffle()
ds = ds.train_test_split(train_size=0.8, stratify_by_column="label")

In [4]:
train_loader = DataLoader(ds["train"], shuffle=True, batch_size=32)
test_loader = DataLoader(ds["test"], shuffle=True, batch_size=32)

In [13]:
class AlzeimersModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) # 1x(128x128) -> 32x(128x128)
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)# 32x(128x128) -> 32x(64x64)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 32x(64x64) -> 64x(64x64)
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2) # 64x(64x64) -> 64x(32x32)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # 64x(32x32) -> 128x(32x32)
        self.relu3 = nn.LeakyReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2) # 128x(32x32) -> 128x(16x16)

        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1) # 128x(16x16) -> 256x(16x16)
        self.relu4 = nn.LeakyReLU()
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2) # 256x(16x16) -> 256x(8x8)

        self.fc1   = nn.Linear(256 * 8 * 8, 256 * 8) # flatten
        self.dropout1 = nn.Dropout(0.5)
        self.fc2   = nn.Linear(256 * 8, 4)

    def forward(self, x):
        # first pass
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        # second pass
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        # third pass
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # fourth pass
        x = self.conv4(x)
        x = self.relu4(x)
        x = self.pool4(x)

        # flatten
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        
        return x

In [14]:
model = AlzeimersModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [15]:
def train_model(model, optimizer, criterion, device, train_loader, epoch):
    model.train()
    for idx, batch in enumerate(train_loader):
        image = batch["image"].to(device)
        target = batch["label"].to(device)
        outputs = model(image)
        loss = criterion(outputs, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if idx % 25 == 0:
            print(f"Epoch [{epoch}].[{idx}] Loss: {loss}")

    with torch.no_grad():
        num_correct = 0
        num_total = 0
        for batch in train_loader:
            image = batch["image"].to(device)
            target = batch["label"].to(device)
            outputs = model(image)
            output = outputs.argmax(dim=1)
            num_correct += (output == target).sum().item()
            num_total += target.size(0)
    accuracy = num_correct / num_total
    print("")
    print(f"Epoch [{epoch}] Train accuracy: {accuracy}")

def test_model(model, device, test_loader, epoch):
    with torch.no_grad():
        num_correct = 0
        num_total = 0
        for batch in test_loader:
            image = batch["image"].to(device)
            target = batch["label"].to(device)
            outputs = model(image)
            output = outputs.argmax(dim=1)
            num_correct += (output == target).sum().item()
            num_total += target.size(0)
        accuracy = num_correct / num_total
        print("")
        print(f"Epoch [{epoch}] Test accuracy: {accuracy}")
        print("")

In [16]:
num_epochs = 20 # hyperparameter
for epoch in range(1, num_epochs + 1):
    train_model(model, optimizer, criterion, device, train_loader, epoch)
    test_model(model, device, test_loader, epoch)

Epoch [1].[0] Loss: 1.388413667678833
Epoch [1].[25] Loss: 1.3148045539855957
Epoch [1].[50] Loss: 1.301229476928711
Epoch [1].[75] Loss: 1.0306642055511475
Epoch [1].[100] Loss: 0.8602398633956909
Epoch [1].[125] Loss: 0.5962968468666077
Epoch [1].[150] Loss: 0.7730485200881958
Epoch [1].[175] Loss: 0.6946854591369629
Epoch [1].[200] Loss: 0.5444226264953613
Epoch [1].[225] Loss: 0.5708288550376892
Epoch [1].[250] Loss: 0.5623915791511536

Epoch [1] Train accuracy: 0.769275161588181

Epoch [1] Test accuracy: 0.755883710198431

Epoch [2].[0] Loss: 0.730221152305603
Epoch [2].[25] Loss: 0.787966787815094
Epoch [2].[50] Loss: 0.5564850568771362
Epoch [2].[75] Loss: 0.49152088165283203
Epoch [2].[100] Loss: 0.4877294600009918
Epoch [2].[125] Loss: 0.3151075839996338
Epoch [2].[150] Loss: 0.5321158170700073
Epoch [2].[175] Loss: 0.5344568490982056
Epoch [2].[200] Loss: 0.31880009174346924
Epoch [2].[225] Loss: 0.46383291482925415
Epoch [2].[250] Loss: 0.3517451584339142

Epoch [2] Train ac

# Reflection
### Results
The best test accuracy was about 99%. The train and test accuracies through the training processes were comparable, with test accuracy trailing behind by about 2% of the train accuracy.

### Improvements Made
The accuracy of the model improved drastically after generating my own train-test splits. In the previous model, the train-test splits used were taken directly from the original dataset on Hugging Face. The dataset's original splits were generally unbalanced, which caused the training process to not generalize well enough and overfit to the train set. By merging the original splits and generating my own, I was able to improve the training process and the model's performance as a whole. The test accuracy increased by 10%, and the train and test accuracies were much closer after each epoch.

### Other Improvements Needed
The outcome of the Alzeimer's CNN could have potentially been better if slight Data Augmentation was implemented (slight rotation, change in contrast and brightness, added blurness/sharpness, etc.), in order to add more noise to generalize the dataset more.

Implementing proper normalization in the data transformation pipeline could have helped the neural network, specifically the optimization algorithm. Normalization is added, but it uses generic means and standard deviations considering the range of the brightness of each pixel. Finding the true mean and standard deviation could slightly improve the overall performance of the model.

A better dataset could also have yielded better results. The [dataset](https://huggingface.co/datasets/yogitamakkar178/mri_scans_alzeimer_detection) used is relatively small (~10.8k entries).

Finally, more experimentation is needed with the hyperparameters and other considerations of the model to determine optimal values for each. These include:
- batch size
- learning rate
- number of epochs
- number of hidden layers
- number of filters for each convolutional layer
- type of activation functions used